In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from glob import glob
import pandas as pd
import re
from pprint import pprint
import pkg_resources

from pymedextcore.document import Document
from pymedext_eds.annotators import Endlines, SentenceTokenizer, Hypothesis, \
                                    ATCDFamille, SyntagmeTokenizer, Negation, RegexMatcher, \
                                    QuickUMLSAnnotator, Pipeline, SectionSplitter, Pipeline

from pymedext_eds.utils import rawtext_loader
from pymedext_eds.ner import NERAnnotator, NERNormalizer

In [3]:
endlines = Endlines(['raw_text'], 'clean_text', 'endlines:v1')
sentences = SentenceTokenizer(['clean_text'], 'sentence', 'sentenceTokenizer:v1')
sections = SectionSplitter(['clean_text'], "section", ID= 'sections')
hypothesis = Hypothesis(['sentence'], 'hypothesis', 'hypothesis:v1')
family = ATCDFamille(['sentence'], 'context', 'ATCDfamily:v1')
syntagmes = SyntagmeTokenizer(['sentence'], 'syntagme', 'SyntagmeTokenizer:v1')
negation = Negation(['syntagme'], 'negation', 'Negation:v1')
regex = RegexMatcher(['clean_text','syntagme'], 'regex', 'RegexMatcher:v1', 'list_regexp.json')

sosy|T184|Sign or Symptom  


dsyn|T047|Disease or Syndrome  
neop|T191|Neoplastic Process  
comd|T049|Cell or Molecular Dysfunction  
mobd|T048|Mental or Behavioral Dysfunction   
patf|T046|Pathologic Function  
anab|T190|Anatomical Abnormality  
cgab|T019|Congenital Abnormality  
acab|T020|Acquired Abnormality  
inpo|T037|Injury or Poisoning  

diap|T060|Diagnostic Procedure  
lbpr|T059|Laboratory Procedure  
lbtr|T034|Laboratory or Test Result  

topp|T061|Therapeutic or Preventive Procedure  


In [4]:
#                                    accepted_semtypes = {'T184', 'T047', 'T191', 'T049', 'T048', 'T046', 'T190', 'T019', 'T020', 'T037','T060',  'T059', 'T034', 'T061'},


In [5]:
quick_umls_th = 0.9
quick_umls_dist = "score"

umls_syntagme = QuickUMLSAnnotator(['syntagme'], 'umls_syntagme', 'QuickUMLS:2020AA', 
                                   quickumls_fp='data/umls2_UL/',
                                   overlapping_criteria=quick_umls_dist,
                                   threshold=quick_umls_th,
                                   similarity_name='jaccard',
                                   accepted_semtypes = {'T184', 'T047', 'T191', 'T049', 'T048', 'T046', 'T190', 'T019', 'T020', 'T037'},
                                   window=5)

umls_signs = QuickUMLSAnnotator(['ENT/SIGNS'], 'umls_signs', 'QuickUMLS:2020AA', 
                                quickumls_fp='data/umls2_UL/',
                                overlapping_criteria=quick_umls_dist,
                                threshold=quick_umls_th,
                                similarity_name='jaccard',
                                accepted_semtypes = {'T184'},
                                window=5)

umls_diag_proc = QuickUMLSAnnotator(['ENT/DIAG_PROC'], 'umls_diag_proc', 'QuickUMLS:2020AA', 
                                quickumls_fp='data/umls2_UL/',
                                overlapping_criteria=quick_umls_dist,
                                threshold=quick_umls_th,
                                similarity_name='jaccard',
                                accepted_semtypes = {'T060', 'T059', 'T034'},
                                window=5)

umls_diag_name = QuickUMLSAnnotator(['ENT/DIAG_NAME'], 'umls_diag_name', 'QuickUMLS:2020AA', 
                                quickumls_fp='data/umls2_UL/',
                                overlapping_criteria=quick_umls_dist,
                                threshold=quick_umls_th,
                                similarity_name='jaccard',
                                accepted_semtypes = {'T047', 'T191', 'T049', 'T048', 'T046', 'T190', 'T019', 'T020', 'T037'},
                                window=5)

umls_therap = QuickUMLSAnnotator(['ENT/THERAP_PROC'], 'umls_therap', 'QuickUMLS:2020AA', 
                                quickumls_fp='data/umls2_UL/',
                                overlapping_criteria=quick_umls_dist,
                                threshold=quick_umls_th,
                                similarity_name='jaccard',
                                accepted_semtypes = {'T061'},
                                window=5)


In [6]:
models_param = [{'tagger_path':'data/models/apcner_deid/entities_7/best-model.pt' ,
                'tag_name': 'pheno_pred' },
                {'tagger_path':'data/models/apmed5/entities/final-model.pt' ,
                'tag_name': 'entity_pred' },
                {'tagger_path':'data/models/apmed5/events/final-model.pt' ,
                'tag_name': 'event_pred' },
               {'tagger_path': "data/models/apmed5/drugblob/final-model.pt",
                'tag_name': 'drugblob_pred'}]

ner = NERAnnotator(['sentence'], 'ner', ID='med:v2', models_param=models_param,  device='cuda:1')

2021-02-23 12:00:44,270 loading file data/models/apcner_deid/entities_7/best-model.pt
2021-02-23 12:00:51,315 loading file data/models/apmed5/entities/final-model.pt
2021-02-23 12:00:54,324 loading file data/models/apmed5/events/final-model.pt
2021-02-23 12:00:57,300 loading file data/models/apmed5/drugblob/final-model.pt


In [7]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/romedi')
romedi_path = glob(data_path + '/*.p')[0]

norm = NERNormalizer(['ENT/DRUG','ENT/CLASS'], 'normalized_mention', ID='norm',romedi_path= romedi_path)

In [8]:
pipeline = [endlines, sections, sentences, hypothesis, family, syntagmes,
            negation, regex, umls_syntagme, ner,  norm, 
            umls_signs, umls_diag_proc, umls_diag_name, umls_therap]

In [13]:
# data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob("data/test_data" + '/*.txt')

docs = [rawtext_loader(x) for x in file_list]

In [14]:
file_list

['data/test_data/11.txt']

In [15]:
for doc in docs:
    doc.annotate(pipeline)

In [18]:
doc.get_annotations("ENT/SIGNS")

In [11]:
ann_list = glob("../../brat_data/covid_pheno_norm/annotator_0_training" + '/*.ann')


In [12]:
def read_brat_ann(path):
    doc_id =  os.path.splitext(os.path.basename(path))[0].split('_')[-1]
    with open(path, "r") as h:
        ann_norm = []
        ann_ner = []
        for line in h.readlines():
            ann_id, field, mention = line.split('\t')
            if ann_id[0] == "T":
                ent_type = field.split(' ')[0]
                start = field.split(' ')[1]
                stop = field.split(' ')[-1]
                ann_ner.append((doc_id, ann_id, ent_type, start, stop, mention.strip()))
            elif ann_id[0] == "N":
                _, ent_id, code = field.split(' ')
                termino, code = code.split(':')
                ann_norm.append((doc_id, ann_id, ent_id, termino, code, mention.strip()))
    
    return(ann_norm, ann_ner)

In [13]:
all_ann = []
for ann_path in ann_list:
    ann_norm, ann_ner = read_brat_ann(ann_path)
    ann_ner = pd.DataFrame(ann_ner, columns=["doc_id", "ent_id", "ent_type", "start", "stop", "mention"])
    ann_norm =  pd.DataFrame(ann_norm, columns = ["doc_id", "ann_id", "ent_id", "termino", "cui", "mention"])
    ann_ner = ann_ner.merge(ann_norm, how= "left", on = ["doc_id", "ent_id"], suffixes=['_ner', '_norm'])
    all_ann.append(ann_ner)

all_ann = pd.concat(all_ann)

In [14]:
all_ann = all_ann.loc[lambda x:x.ent_type.isin(["DIAG_NAME", "SIGNS"])]

In [15]:
all_ann.assign(has_norm = lambda x:x.cui.notnull()).groupby(['ent_type'])['has_norm'].aggregate([sum, len, lambda x: sum(x)/len(x)])

,sum,len,<lambda_0>
ent_type,,,
DIAG_NAME,106,126,0.841270
SIGNS,127,180,0.705556


In [16]:
all_ann = all_ann.loc[lambda x:x.cui.notnull()]

In [17]:
all_ann.head()

,doc_id,ent_id,ent_type,start,stop,mention_ner,ann_id,termino,cui,mention_norm
7,698430240,T25,SIGNS,1510,1547,douleurs intenses notamment nocturnes,N13,UMLS_FR,C0030193,Douleur SAI
8,698430240,T1,SIGNS,201,208,malaise,N1,UMLS_FR,C0231218,Se sentait malade en général
9,698430240,T2,SIGNS,292,328,contusion directe de l’épaule droite,N2,UMLS_FR,C2218555,
10,698430240,T3,SIGNS,330,357,impotence massive immédiate,N3,UMLS_FR,C0311394,Incapacité de marcher
11,698430240,T4,DIAG_NAME,395,437,rupture étendue au niveau du supra-épineux,N4,UMLS_FR,C0439059,


In [19]:
# pd.DataFrame.from_records(NERAnnotator.doc_to_omop(docs[1]))

In [20]:
def to_pdf(doc, key):
    columns = ["doc_id", "mention", "start", "end", "cui", "label"]
    lines = [(doc.source_ID, t.value, t.span[0], t.span[1], t.attributes["cui"], t.attributes["label"]) \
             for t in doc.get_annotations(key)]
    
    
    df = pd.DataFrame(lines if lines else {k: [] for k in columns}, 
                      columns = columns
                     )    
    return df

In [21]:
ner_acc = []
syn_acc = []

for doc in docs:
    df_syn = to_pdf(doc, 'umls_syntagme')
    
#     ner_acc.append(to_pdf(doc, 'umls_therap'))
    ner_acc.append(to_pdf(doc, 'umls_diag_name'))
#     ner_acc.append(to_pdf(doc, 'umls_diag_proc'))
    ner_acc.append(to_pdf(doc, 'umls_signs'))

    syn_acc.append(df_syn)

ner_acc = pd.concat(ner_acc).assign(mod = "ner")
syn_acc = pd.concat(syn_acc).assign(mod = "syn")

In [22]:
ner_acc.head()

,doc_id,mention,start,end,cui,label,mod
0,698430240,contusion,297.0,306.0,C0009938,contusion,ner
1,698430240,rupture,401.0,408.0,C3203359,rupture,ner
2,698430240,rupture,446.0,453.0,C3203359,rupture,ner
3,698430240,épanchement intra-articulaire assez,484.0,519.0,C1253936,epanchement intra-articulair,ner
4,698430240,traumatisme,1429.0,1440.0,C3714660,traumatisme,ner


In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score

In [24]:
def compute_metrics(gold, pred, label = ""):
    tp, fn, fp, tn = confusion_matrix(gold, pred).ravel()
    acc = accuracy_score(gold, pred)
    f1= f1_score(gold, pred, average="binary")
    r = recall_score(gold, pred, average="binary")
    p = precision_score(gold, pred, average="binary")
    
    #reformat metrics
    metrics = pd.DataFrame([[tp + fn, tp, fn, fp, r, p, f1, acc]],
                           columns=['total_pos', "TP", "FN", "FP", "Recall", "Precision", "F1", "Accuracy"], index = [label])
    return metrics

In [25]:
all_merge = (all_ann
             .merge(ner_acc, how = "outer", on =["doc_id", "cui"])
             .merge(syn_acc, how = "outer", on =["doc_id", "cui"])
             .assign(true = lambda x:x.ent_id.notnull())
             .assign(pred_ner = lambda x:x['mod_x'].notnull())
             .assign(pred_q = lambda x:x['mod_y'].notnull())

            )

In [26]:
pd.concat([compute_metrics(all_merge.true, all_merge.pred_ner, label = "NER + QuickUMLS (ent level)"), 
           compute_metrics(all_merge.true, all_merge.pred_q, label = "QuickUMLS (ent level)"),
           compute_metrics(all_merge.drop_duplicates(["doc_id", "cui"]).true, all_merge.drop_duplicates(["doc_id", "cui"]).pred_ner, label = "NER + QuickUMLS (doc level)"),
           compute_metrics(all_merge.drop_duplicates(["doc_id", "cui"]).true, all_merge.drop_duplicates(["doc_id", "cui"]).pred_q, label = "QuickUMLS (doc level)")]
         ).round(2)

,total_pos,TP,FN,FP,Recall,Precision,F1,Accuracy
NER + QuickUMLS (ent level),168,73,95,162,0.78,0.86,0.81,0.71
QuickUMLS (ent level),168,3,165,140,0.81,0.78,0.79,0.66
NER + QuickUMLS (doc level),104,55,49,122,0.29,0.51,0.37,0.38
QuickUMLS (doc level),104,3,101,110,0.36,0.38,0.37,0.24


In [15]:
len(syn_acc), len(ner_acc)

(682, 418)

In [16]:
ner_acc.merge(syn_acc, how = "inner", on =["doc_id", "start", "end", "cui"]).shape

(296, 10)

In [17]:
diff = ner_acc.merge(syn_acc, how = "outer", on =["doc_id", "start", "end"]).loc[lambda x:x.cui_x != x.cui_y]

In [18]:
from IPython.display import display_html

In [19]:
gen = ((k, group) for k, group in diff.groupby('doc_id'))


In [21]:
k, group = next(gen)
ner_mention = group.loc[lambda x:x.mod_x.notnull(), ["mention_x", "label_x"]]
syn_mention = group.loc[lambda x:x.mod_y.notnull(), ["mention_y", "label_y"]]

display_html(ner_mention)
display_html(syn_mention)

,mention_x,label_x
19,cardiopathie ischémique,myelopathie ischemique
20,infarctus,infarctus
31,échographie cardiaque,echographie cardiaque
33,cardiovasculaire,syncope cardiovasculaire


,mention_y,label_y
33,cardiovasculaire,nevrose cardiovasculaire
450,consultation,teleconsultation
451,consultation,teleconsultation
452,suivi de cardiopathie ischémique,cardiopathie ischemique
453,fait infarctus,post infarctus
454,TSA et des membres inférieurs,ulceres des membres inferieurs
455,fraction d’éjection retrouvée,fraction d'ejection
456,échographie,echographie sai
457,consultation,teleconsultation
458,chute,chutes


In [ ]:
# python -m quickumls.install -E FR data/umls data/quickumls

In [19]:
from pymedext_eds.viz import display_annotations

In [20]:
doc_demo = rawtext_loader("data/test_data/11.txt") 

In [21]:
doc_demo.annotate(pipeline)

In [22]:
display_annotations(doc_demo, entities=['ENT/DIAG_NAME', 'ENT/SIGNS'])

In [23]:
display_annotations(doc_demo, entities=['umls_syntagme'])

In [214]:
display_annotations(doc_demo, entities=['ENT/SIGNS'])

In [215]:
display_annotations(doc_demo, entities=['ENT/DIAG_NAME'])